# Modeling and Simulation in Python

Chapter 9

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [ ]:
# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import everything from SymPy.
from sympy import *

# Set up Jupyter notebook to display math.
init_printing() 

The following displays SymPy expressions and provides the option of showing results in LaTeX format.

In [ ]:
from sympy.printing import latex

def show(expr, show_latex=False):
    """Display a SymPy expression.
    
    expr: SymPy expression
    show_latex: boolean
    """
    if show_latex:
        print(latex(expr))
    return expr

### Analysis with SymPy

Create a symbol for time.

In [1]:
import sympy as sp

t = sp.symbols('t')
t

t

If you combine symbols and numbers, you get symbolic expressions.

In [2]:
expr = t + 1

The result is an `Add` object, which just represents the sum without trying to compute it.

In [3]:
type(expr)

sympy.core.add.Add

`subs` can be used to replace a symbol with a number, which allows the addition to proceed.

In [4]:
expr.subs(t, 2)

3

`f` is a special class of symbol that represents a function.

In [6]:
f = sp.Function('f')

The type of `f` is `UndefinedFunction`

In [7]:
type(f)

sympy.core.function.UndefinedFunction

SymPy understands that `f(t)` means `f` evaluated at `t`, but it doesn't try to evaluate it yet.

In [8]:
f(t)

f(t)

`diff` returns a `Derivative` object that represents the time derivative of `f`

In [9]:
dfdt = sp.diff(f(t), t)

In [10]:
type(dfdt)

sympy.core.function.Derivative

We need a symbol for `alpha`

In [11]:
alpha = sp.symbols('alpha')

Now we can write the differential equation for proportional growth.

In [12]:
eq1 = sp.Eq(dfdt, alpha*f(t))

And use `dsolve` to solve it.  The result is the general solution.

In [13]:
solution_eq = sp.dsolve(eq1)

We can tell it's a general solution because it contains an unspecified constant, `C1`.

In this example, finding the particular solution is easy: we just replace `C1` with `p_0`

In [14]:
C1, p_0 = sp.symbols('C1 p_0')

In [15]:
particular = solution_eq.subs(C1, p_0)

In the next example, we have to work a little harder to find the particular solution.

### Solving the quadratic growth equation 

We'll use the (r, K) parameterization, so we'll need two more symbols:

In [16]:
r, K = sp.symbols('r K')

Now we can write the differential equation.

In [18]:
eq2 = sp.Eq(sp.diff(f(t), t), r * f(t) * (1 - f(t)/K))

And solve it.

In [19]:
solution_eq = sp.dsolve(eq2)

The result, `solution_eq`, contains `rhs`, which is the right-hand side of the solution.

In [20]:
general = solution_eq.rhs

We can evaluate the right-hand side at $t=0$

In [21]:
at_0 = general.subs(t, 0)

Now we want to find the value of `C1` that makes `f(0) = p_0`.

So we'll create the equation `at_0 = p_0` and solve for `C1`.  Because this is just an algebraic identity, not a differential equation, we use `solve`, not `dsolve`.

The result from `solve` is a list of solutions.  In this case, [we have reason to expect only one solution](https://en.wikipedia.org/wiki/Picard%E2%80%93Lindel%C3%B6f_theorem), but we still get a list, so we have to use the bracket operator, `[0]`, to select the first one.

In [23]:
solutions = sp.solve(sp.Eq(at_0, p_0), C1)
type(solutions), len(solutions)

(list, 1)

In [24]:
value_of_C1 = solutions[0]

Now in the general solution, we want to replace `C1` with the value of `C1` we just figured out.

In [25]:
particular = general.subs(C1, value_of_C1)

The result is complicated, but SymPy provides a method that tries to simplify it.

In [26]:
particular = sp.simplify(particular)

Often simplicity is in the eye of the beholder, but that's about as simple as this expression gets.

Just to double-check, we can evaluate it at `t=0` and confirm that we get `p_0`

In [27]:
particular.subs(t, 0)

p_0

This solution is called the [logistic function](https://en.wikipedia.org/wiki/Population_growth#Logistic_equation).

In some places you'll see it written in a different form:

$f(t) = \frac{K}{1 + A e^{-rt}}$

where $A = (K - p_0) / p_0$.

We can use SymPy to confirm that these two forms are equivalent.  First we represent the alternative version of the logistic function:

In [28]:
A = (K - p_0) / p_0

In [29]:
logistic = K / (1 + A * sp.exp(-r*t))

To see whether two expressions are equivalent, we can check whether their difference simplifies to 0.

In [30]:
sp.simplify(particular - logistic)

0

This test only works one way: if SymPy says the difference reduces to 0, the expressions are definitely equivalent (and not just numerically close).

But if SymPy can't find a way to simplify the result to 0, that doesn't necessarily mean there isn't one.  Testing whether two expressions are equivalent is a surprisingly hard problem; in fact, there is no algorithm that can solve it in general.

### Exercises

**Exercise:** Solve the quadratic growth equation using the alternative parameterization

$\frac{df(t)}{dt} = \alpha f(t) + \beta f^2(t) $

In [31]:
# Solution goes here
alpha, beta = sp.symbols('alpha beta')

In [32]:
# Solution goes here
eq3 = sp.Eq(sp.diff(f(t), t), alpha*f(t) + beta*f(t)**2)


In [33]:
# Solution goes here
solution_eq = sp.dsolve(eq3)

In [34]:
# Solution goes here
general = solution_eq.rhs

In [35]:
# Solution goes here
at_0 = general.subs(t, 0)


In [36]:
# Solution goes here
solutions = sp.solve(sp.Eq(at_0, p_0), C1)
value_of_C1 = solutions[0]

In [39]:
# Solution goes here
particular = general.subs(C1, value_of_C1)
sp.simplify(particular)

alpha*p_0*exp(alpha*t)/(alpha - beta*p_0*exp(alpha*t) + beta*p_0)

**Exercise:**  Use [WolframAlpha](https://www.wolframalpha.com/) to solve the quadratic growth model, using either or both forms of parameterization:

    df(t) / dt = alpha f(t) + beta f(t)^2

or

    df(t) / dt = r f(t) (1 - f(t)/K)

Find the general solution and also the particular solution where `f(0) = p_0`.